In [ ]:
import time
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(20,5))

In [ ]:
df = pd.read_pickle('../data/shippify_icd.pkl', compression='gzip')
df.info()

### **Agregar codigo necesario debajo de su nombre**

## Enmanuel Magallanes

In [ ]:
## Question: cual son las franjas horarias en las que se regitran mas tareas atrasadas y de que (tipo|peso) son?

## Josue Cobos

In [ ]:
## Question: cual son las franjas horarias en las que se regitran mas tareas atrasadas y de que (tipo|peso) son?